Shows how one can generate text given a prompt and some hyperparameters, using either minGPT or huggingface/transformers

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

In [2]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2-xl'
device = 'cuda'

In [3]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

number of parameters: 1557.61M


In [4]:

def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [5]:
generate(prompt='Andrej Karpathy, the', num_samples=10, steps=20)

--------------------------------------------------------------------------------
Andrej Karpathy, the chief of the criminal investigation department, said during a news conference, "We still have a lot of
--------------------------------------------------------------------------------
Andrej Karpathy, the man whom most of America believes is the architect of the current financial crisis. He runs the National Council
--------------------------------------------------------------------------------
Andrej Karpathy, the head of the Department for Regional Reform of Bulgaria and an MP in the centre-right GERB party
--------------------------------------------------------------------------------
Andrej Karpathy, the former head of the World Bank's IMF department, who worked closely with the IMF. The IMF had
--------------------------------------------------------------------------------
Andrej Karpathy, the vice president for innovation and research at Citi who oversaw the team's work to mak

In [ ]:
import torch
from mingpt.bpe import BPETokenizer
from mingpt.model import GPT
import matplotlib.pyplot as plt
import numpy as np

# Inicializar el modelo GPT y el tokenizer
model = GPT.from_pretrained('gpt2')  # Asegúrate de cargar el modelo correcto
model.eval()  # Modo evaluación
bpe = BPETokenizer()

# Texto limpio y tokenización
clean_text = "Michelle Jones was a top-notch student. Michelle"
print(f"Clean text: {clean_text}")
tokens_clean = bpe(clean_text)[0]  # Tokeniza el texto
input_length = tokens_clean.size(0)

    # Ejecutar el modelo en el texto limpio
with torch.no_grad():
    logits, _ = model(tokens_clean.unsqueeze(0))  # Guarda activaciones
    last_logits = logits[0, -1, :]  # Logits del último token
    print(f"Logits shape: {last_logits.shape}")

# Mostrar continuaciones más probables
probabilities = torch.softmax(last_logits, dim=-1)
top_k = 20  # Número de continuaciones más probables a mostrar
top_indices = torch.topk(probabilities, k=top_k).indices
top_probabilities = probabilities[top_indices]

print(f"Top {top_k} continuations:")
for i, index in enumerate(top_indices):
    token_str = bpe.decode(torch.tensor([index.item()]))  # Decodificar token
    prob = top_probabilities[i].item()
    print(f"{i + 1}: {token_str} ({prob:.4f})")

# Ejemplo para obtener la probabilidad de un token específico (e.g., "Jones")
token_to_check = " Jones"
index_to_check = bpe(token_to_check)[0][0].item()
prob_of_token = probabilities[index_to_check].item()
print(f"Probability of '{token_to_check.strip()}': {prob_of_token:.4f}")

corrupt_text = "Michelle Smith was a top-notch student. Michelle"

In [ ]:
def analyze_embeddings(model, tokenizer, clean_text, corrupted_text, target_tokens):
    """
    Analyze the impact of embeddings on logits for a given corrupted text.

    Args:
        model: GPT model.
        tokenizer: BPE tokenizer.
        clean_text (str): Clean sequence.
        corrupted_text (str): Corrupted sequence.
        target_tokens (list): Tokens to analyze logits for (e.g., ["Jones", "Smith"]).

    Returns:
        None
    """
    # Tokenize texts
    clean_tokens = tokenizer(clean_text)[0]
    corrupted_tokens = tokenizer(corrupted_text)[0]
    input_tensor = corrupted_tokens.unsqueeze(0)

    # Process clean sequence
    model.eval()
    with torch.no_grad():
        model.forward(clean_tokens.unsqueeze(0), save_embeddings=True)
        clean_logits = model.stored_logits  # Save logits from the clean sequence
        print(f"Logits shape: {clean_logits.shape}")

    # Mostrar continuaciones más probables
    probabilities = torch.softmax(clean_logits, dim=-1)
    top_k = 20  # Número de continuaciones más probables a mostrar
    top_indices = torch.topk(probabilities, k=top_k).indices[0]
    top_probabilities = probabilities[0,top_indices]

    print(f"Top {top_k} continuations:")
    for i, index in enumerate(top_indices):
        token_str = bpe.decode(torch.tensor([index.item()]))  # Decodificar token
        prob = top_probabilities[i].item()
        print(f"{i + 1}: {token_str} ({prob:.4f})")

    # Get target token indices
    target_indices = [tokenizer(f" {token}")[0][0].item() for token in target_tokens]

    # Initialize difference matrix
    n_layers = len(model.transformer.h)
    seq_len = input_tensor.size(1)
    difference_matrix = np.zeros((n_layers, seq_len))

    # Loop over layers and positions
    for layer in range(n_layers):
        for position in range(seq_len):
            # Replace corrupted embedding with the clean embedding
            intervened_embedding = model.stored_embeddings[layer][position]
            model.forward(input_tensor, intervene_at=(layer, position, intervened_embedding))

            # Get the logits from the intervened model
            corrupted_logits = model.stored_logits

            # Calculate the difference between clean and intervened logits for the target token
            logit_diff = (corrupted_logits[0, target_indices[0]] - corrupted_logits[0, target_indices[1]]).item()
            difference_matrix[layer, position] = logit_diff

    # Visualize heatmap without the first row
    plt.figure(figsize=(10, 8))  # Ensure a clean figure
    plt.matshow(difference_matrix[1:, :], cmap="Greys_r", fignum=False)  # Exclude the first row
    plt.colorbar(label="Logit Difference")
    plt.title(f"Logit Differences for Tokens: {', '.join(target_tokens)}")
    plt.xlabel("Position in Sequence")
    plt.ylabel("Layer")
    plt.show()



In [ ]:
clean_text = "Michelle Jones was a top-notch student. Michelle"
corrupted_text = "Michelle Smith was a top-notch student. Michelle"
target_tokens = ["Smith", "Jones"]

# Analyze embeddings
analyze_embeddings(model, bpe, clean_text, corrupted_text, target_tokens)

In [ ]:
clean_text = "Jessica Jones was a top-notch student. Michelle"
corrupted_text = "Michelle Smith was a top-notch student. Jessica"
target_tokens = ["Smith", "Jones"]

# Analyze embeddings
analyze_embeddings(model, bpe, clean_text, corrupted_text, target_tokens)

In [ ]:
clean_text = "Messi is the best player. Messi"
corrupted_text = "Messi is the worst player. Messi"
target_tokens = ["worst", "best"]

# Analyze embeddings
analyze_embeddings(model, bpe, clean_text, corrupted_text, target_tokens)

In [ ]:
clean_text = "Once upon a time, there was a princess"
corrupted_text = "Once upon a time, there was a dragon"
target_tokens = ["dragon","princess"]

# Analyze embeddings
analyze_embeddings(model, bpe, clean_text, corrupted_text, target_tokens)

In [ ]:
clean_text = "Thomas Green was an excellent musician. Thomas"
corrupted_text = "Thomas White was an excellent musician. Thomas"
target_tokens = ["White", "Green"]

# Analyze embeddings
analyze_embeddings(model, bpe, clean_text, corrupted_text, target_tokens)

In [ ]:
clean_text = "Thomas Green was an excellent musician. Thomas"
corrupted_text = "Thomas Green was an excellent magician. Thomas"
target_tokens = ["musician", "magician"]

# Analyze embeddings
analyze_embeddings(model, bpe, clean_text, corrupted_text, target_tokens)

In [ ]:
clean_text = "Albert Green was an excellent musician. Thomas"
corrupted_text = "Thomas Green was an excellent musician. Albert"
target_tokens = ["Albert", "Thomas"]

# Analyze embeddings
analyze_embeddings(model, bpe, clean_text, corrupted_text, target_tokens)

In [ ]:
clean_text = "I went to the bank to deposit money. The bank"
corrupted_text = "I went to the bank to catch fish.The bank"
target_tokens = ["catch", "deposit"]

# Analyze embeddings
analyze_embeddings(model, bpe, clean_text, corrupted_text, target_tokens)